#**Система распознавания русскоязычного текста, сгенерированного LLM**

#Установка и загрузка библиотек и ресурсов

In [1]:
!pip -q install pymorphy2
!pip -q install sentencepiece

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 47.9 MB/s eta 0:00:00


In [2]:
!pip -q install spacy

In [3]:
!python -m spacy download ru_core_news_sm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 82.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import pandas as pd
import numpy as np
import sklearn

In [5]:
from sklearn.model_selection import train_test_split # делим на тестовую и тренировочную
from sklearn.feature_extraction.text import CountVectorizer # мешок слов
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report # метрика точности, f1 мера, матрица ошибок и отчёт по классификации
from sklearn import model_selection # выбор модели и оценка параметров (можно загрузить несколько моделей)
from sklearn.linear_model import LogisticRegression # логистическая регрессия
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # бустинг, случайный лес
from sklearn.svm import SVC # метод SVM, загружаем с ядром rbf https://scikit-learn.ru/1-4-support-vector-machines/
from sklearn.neighbors import KNeighborsClassifier # к ближайших соседей

In [6]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pymorphy2
import re

In [7]:
import spacy
import math
from collections import Counter
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [9]:
# Загрузка необходимых ресурсов
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('ru_core_news_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
import pickle
import joblib
from sklearn.base import BaseEstimator, TransformerMixin

#Загрузка моделей

**Модели обученные на TF-IDF признаках**

GB_model.pkl,

kNN_model.pkl,

LogisticRegression_model.pkl,

RandomForest_model.pkl,

SVC_model.pkl

vectorizer.pkl - **TF-IDF векторизатор**

=========================================================

**Модели обученные на новых числовых признаках**

GB1_model1.pkl,

kNN1_model1.pkl,

LogisticRegression1_model1.pkl,

RandomForest1_model1.pkl,

SVC1_model1.pkl

In [11]:
#Загрузка моделей
!gdown 1RJ5Ig8pBgzTPiXr12HJRaNrKtaU09WtT
!gdown 1x0vxLSmwPr4UjR_S9_o7h0l9plafmrCB
!gdown 1ODMLAfqvTT6KNlbKurX78rFLeMXP--tn
!gdown 1HGctCTC0xkYfsUdbtiE5PO4tYMIFJBRm
!gdown 1_w9tpM1VSqNm5rVqV-4Awvlua1eTgEp-
!gdown 1Ppc0L49v_i5v0MNl9esb0Ta1H695VEQW
!gdown 1cRf6z_QMLVG9ki5YOSWQLhneNLGHXn1Q
!gdown 14K6e9PLU3CfdSu-QRVDPRgDmcBXj3vxP
!gdown 1qvzMkRpP-17bPCANflJQgUz_HgTtxlWB
!gdown 1NyKrUZ1-GUiNfpj5yBHylwiAgD9f-rqG
!gdown 1QDS8s7Xoud0zE9jhqMWD41OrqNpp9R7t

Downloading...
From: https://drive.google.com/uc?id=1RJ5Ig8pBgzTPiXr12HJRaNrKtaU09WtT
To: /content/GB_model.pkl
100% 135k/135k [00:00<00:00, 5.39MB/s]
Downloading...
From: https://drive.google.com/uc?id=1x0vxLSmwPr4UjR_S9_o7h0l9plafmrCB
To: /content/GB1_model1.pkl
100% 135k/135k [00:00<00:00, 10.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ODMLAfqvTT6KNlbKurX78rFLeMXP--tn
To: /content/kNN_model.pkl
100% 2.63M/2.63M [00:00<00:00, 18.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HGctCTC0xkYfsUdbtiE5PO4tYMIFJBRm
To: /content/kNN1_model1.pkl
100% 1.33M/1.33M [00:00<00:00, 19.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_w9tpM1VSqNm5rVqV-4Awvlua1eTgEp-
To: /content/LogisticRegression_model.pkl
100% 122k/122k [00:00<00:00, 10.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ppc0L49v_i5v0MNl9esb0Ta1H695VEQW
To: /content/LogisticRegression1_model1.pkl
100% 2.83k/2.83k [00:00<00:00, 9.73MB/s]
Downloading...
From: https://drive.google.com/uc

In [12]:
# Загрузка стоп-слов и инициализация лемматизатора
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Функции очистки текста, загрузки моделей на TF-IDF признаках и векторизатора**

In [13]:
# Функция для предобработки текста
def preprocess_text(text):
    # Приведение к нижнему регистру и удаление пунктуации
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация и лемматизация
    tokens = text.split()
    lemmatized = [morph.parse(token)[0].normal_form for token in tokens if token not in russian_stopwords]
    return ' '.join(lemmatized)

In [14]:

def get_models():
    return {
        'RandomForest': RandomForestClassifier(bootstrap=True, n_estimators=100, n_jobs=-1),
        "GB": GradientBoostingClassifier(n_estimators=100),
        "kNN": KNeighborsClassifier(),
        "LogisticRegression": LogisticRegression(),
        "SVC": SVC(kernel='rbf')
    }

"""
# Функция для сохранения моделей и векторизатора
def save_models(models, vectorizer):
    for name, model in models.items():
        with open(f'{name}_model.pkl', 'wb') as f:
            pickle.dump(model, f)

    with open('vectorizer.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)
"""
# Функция для загрузки моделей и векторизатора
def load_models():
    loaded_models = {}
    model_names = ['RandomForest', 'GB', 'kNN', 'LogisticRegression', 'SVC']

    for name in model_names:
        with open(f'{name}_model.pkl', 'rb') as f:
            loaded_models[name] = pickle.load(f)

    with open('vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

    return loaded_models, vectorizer



**Проверяем загрузку моделей и возможность классификации**

In [15]:
# Пример использования сохраненных моделей
loaded_models, loaded_vectorizer = load_models()

In [16]:
paragraph = "Пользователи, или клиенты, могут отправлять, получать, просматривать и управлять документами через специальные приложения, которые взаимодействуют с сервером ис учёта ресурсов. Таким образом, ис учёта ресурсов обычно использует клиентсерверную архитектуру, где сервер обрабатывает запросы на поставку и хранение материалов, а клиенты работают с этой системой через свои приложения. Клиентсерверная архитектура в ис учёта ресурсов предприятия обеспечивает удобство и эффективность для пользователей. Клиенты могут использовать специализированные приложения для взаимодействия с сервером, отправлять. Сервер, в свою очередь, обеспечивает централизованное хранение данных, обработку запросов, контроль доступа и безопасность информации."

In [17]:
preprocessed_paragraph = preprocess_text(paragraph)

In [18]:
# Вывод результата для проверки
print(preprocessed_paragraph)

пользователь клиент мочь отправлять получать просматривать управлять документ специальный приложение который взаимодействовать сервер иса учёт ресурс такой образ иса учёт ресурс обычно использовать клиентсерверный архитектура сервер обрабатывать запрос поставка хранение материал клиент работать система свой приложение клиентсерверный архитектура иса учёт ресурс предприятие обеспечивать удобство эффективность пользователь клиент мочь использовать специализировать приложение взаимодействие сервер отправлять сервер очередь обеспечивать централизовать хранение данные обработка запрос контроль доступ безопасность информация


In [19]:
# Предсказание на новых данных
new_data = [preprocessed_paragraph]
new_data_vectorized = loaded_vectorizer.transform(new_data)

In [20]:
for name, model in loaded_models.items():
    prediction = model.predict(new_data_vectorized)
    print(f"Предсказание модели {name}: {prediction}")

Предсказание модели RandomForest: [1]
Предсказание модели GB: [1]
Предсказание модели kNN: [1]
Предсказание модели LogisticRegression: [1]
Предсказание модели SVC: [1]


**Конец проверки работоспособности моделей на TF-IDF признаках** ________________

#Работаем с моделями на новых числовых признаках

**Функции для расчета новых числовых признаков**

In [21]:
# Функция для подсчета энтропии распределения символов
def calculate_entropy(text):
    freq = Counter(text)
    entropy = 0
    for char, count in freq.items():
        prob = count / len(text)
        entropy -= prob * math.log2(prob)
    return entropy

# Функция для подсчета энтропии распределения слов
def calculate_word_entropy(tokens):
    freq = Counter(tokens)
    total_words = len(tokens)
    entropy = 0
    for word, count in freq.items():
        prob = count / total_words
        entropy -= prob * math.log2(prob)
    return entropy


# Функция для подсчета TTR
def calculate_ttr(tokens):
    return len(set(tokens)) / len(tokens)

# Функция для подсчета индекса Хердана
def calculate_herdan_index(tokens):
    return math.log(len(set(tokens))) / math.log(len(tokens))


# Функция для подсчета индекса Юла
def calculate_yule_index(tokens):
    freq = Counter(tokens)
    m1 = sum(freq.values())
    freq_of_freq = Counter(freq.values())
    m2 = sum([f * (f - 1) * freq_of_freq[f] for f in freq_of_freq])
    return 10000 * (m2) / (m1 * m1 - m1)


# Функция для подсчета частоты использования служебных слов
def calculate_function_words_ratio(tokens):
    stop_words = set(stopwords.words('russian'))
    return sum(1 for token in tokens if token.lower() in stop_words) / len(tokens)

# Функция для подсчета средней глубины синтаксического дерева
def calculate_avg_tree_depth(doc):
    def get_depth(token):
        depth = 0
        while token.head != token:
            token = token.head
            depth += 1
        return depth

    sentence_depths = []
    for sent in doc.sents:
        depths = [get_depth(token) for token in sent if token.dep_ != 'ROOT']
        if depths:
            sentence_depths.append(sum(depths) / len(depths))

    return sum(sentence_depths) / len(sentence_depths) if sentence_depths else 0


# Функция для подсчета соотношения частей речи
def calculate_pos_ratio(doc):
    pos_counts = Counter(token.pos_ for token in doc)
    total = sum(pos_counts.values())
    if total == 0:
        return {}  # Возвращаем пустой словарь, если текст пустой

    # Создаем словарь со всеми возможными частями речи, инициализируя их нулями
    all_pos = {pos: 0 for pos in spacy.parts_of_speech.IDS}

    # Обновляем значения для найденных частей речи
    for pos, count in pos_counts.items():
        all_pos[pos] = count / total

    return all_pos


#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Функция для подсчета частоты использования падежей - почему-то не работает (!!!)
def calculate_case_frequency(doc):
    case_counts = Counter(token.morph.get('Case') for token in doc if 'Case' in token.morph)
    total = sum(case_counts.values())
    return {case: count / total for case, count in case_counts.items()}

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
# Функция для подсчета частоты биграмм и триграмм - исключила - дает слишком много малоинформативных признаков (!!!)
def calculate_ngram_frequency(tokens, n, top_n=2):
    ngram_counts = Counter(ngrams(tokens, n))
    total = sum(ngram_counts.values())
    return {f'top_{i+1}_{n}gram': ' '.join(ngram) for i, (ngram, _) in enumerate(ngram_counts.most_common(top_n))}


# Функции для расчета индекса удобочитаемости Флеша-Кинкейда (адаптированная для русского языка)
#функция count_syllables_ru для подсчета слогов в русских словах
def count_syllables(word):
    vowels = 'аеёиоуыэюя'
    return len([char for char in word.lower() if char in vowels])

def calculate_flesch_kincaid_grade(text):
    # Очистка текста
    text = re.sub(r'[^\w\s.,!?]', '', text)

    sentences = sent_tokenize(text)
    words = [word.strip('.,!?:;()[]{}') for word in word_tokenize(text) if word.strip('.,!?:;()[]{}')]

    if not sentences or not words:
        return 0

    avg_sentence_length = len(words) / len(sentences)
    avg_syllables_per_word = sum(count_syllables(word) for word in words) / len(words)

    # Адаптированные коэффициенты для русского языка
    score = 220.755 - (1.1 * avg_sentence_length) - (65.0 * avg_syllables_per_word)

    # Ограничение минимального значения
    return max(0, score)


# Функция для расчета индекса туманности Ганнинга (адаптированная для русского языка)
def calculate_gunning_fog_index(text):
    if not text:
        return 0

    # Очистка текста
    text = re.sub(r'[^\w\s]', '', text.lower())

    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    if len(sentences) == 0 or len(words) == 0:
        return 0

    complex_words = sum(1 for word in words if count_syllables(word) >= 4)

    return 0.4 * ((len(words) / len(sentences)) + 100 * (complex_words / len(words)))

# Функция для подсчета частоты повторения слов
def calculate_word_repetition(tokens):
    freq = Counter(tokens)
    return sum(count for count in freq.values() if count > 1) / len(tokens)

**Сам рассчет числовых признаков для датафрейма**

In [22]:
# Применение функций к датафрейму
def extract_features(df):
    features = []
    for _, row in df.iterrows():
        text = row['content']
        tokens = word_tokenize(text.lower())
        sentences = sent_tokenize(text)
        doc = nlp(text)

        feature = {
            'char_count': len(text),
            'word_count': len(tokens),
            'sentence_count': len(sentences),
            'avg_word_length': sum(len(word) for word in tokens) / len(tokens),
            'avg_sentence_length': len(tokens) / len(sentences),
            'ttr': calculate_ttr(tokens),
            'herdan_index': calculate_herdan_index(tokens),
            'yule_index': calculate_yule_index(tokens),
            'function_words_ratio': calculate_function_words_ratio(tokens),
            'avg_punctuation_per_sentence': sum(1 for char in text if char in '.,;:!?') / len(sentences),
            'avg_tree_depth': calculate_avg_tree_depth(doc),
            'char_entropy': calculate_entropy(text),
            #'word_entropy': calculate_entropy(' '.join(tokens)),
            'word_entropy': calculate_word_entropy(tokens),
            'flesch_kincaid_grade': calculate_flesch_kincaid_grade(text),
            'gunning_fog_index': calculate_gunning_fog_index(text),
            'word_repetition_frequency': calculate_word_repetition(tokens),
        }

        pos_ratio = calculate_pos_ratio(doc)
        feature.update({f'pos_ratio_{pos}': ratio for pos, ratio in pos_ratio.items()})

        case_frequency = calculate_case_frequency(doc)
        feature.update({f'case_frequency_{case}': freq for case, freq in case_frequency.items()})

        #bigram_frequency = calculate_ngram_frequency(tokens, 2)
        #feature.update({f'bigram_frequency_{bigram}': freq for bigram, freq in bigram_frequency.items()})

        #trigram_frequency = calculate_ngram_frequency(tokens, 3)
        #feature.update({f'trigram_frequency_{trigram}': freq for trigram, freq in trigram_frequency.items()})

        features.append(feature)

    return pd.DataFrame(features)


In [23]:
def get_models1():
    return {
        'RandomForest1': RandomForestClassifier(bootstrap=True, n_estimators=100, n_jobs=-1),
        "GB1": GradientBoostingClassifier(n_estimators=100),
        "kNN1": Pipeline([
            ('scaler', StandardScaler()),
            ('knn', KNeighborsClassifier())
        ]),
        "LogisticRegression1": Pipeline([
            ('scaler', StandardScaler()),
            ('lr', LogisticRegression())
        ]),
        "SVC1": Pipeline([
            ('scaler', StandardScaler()),
            ('svc', SVC(kernel='rbf'))
        ])
    }


In [24]:
# Класс для извлечения признаков
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Инициализация необходимых ресурсов
        nltk.download('punkt')
        nltk.download('stopwords')
        self.nlp = spacy.load('ru_core_news_sm')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return extract_features(pd.DataFrame({'content': X}))
"""
# Сохранение моделей
def save_models1(models):
    for name, model in models.items():
        filename = f"{name}_model1.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(model, file)
    print("Модели сохранены.")
"""

# Загрузка моделей
def load_models1():
    loaded_models = {}
    for name in ['RandomForest1', 'GB1', 'kNN1', 'LogisticRegression1', 'SVC1']:
        filename = f"{name}_model1.pkl"
        with open(filename, 'rb') as file:
            loaded_models[name] = pickle.load(file)
    print("Модели загружены.")
    return loaded_models

# Функция для классификации произвольного абзаца
def classify_paragraph(paragraph, models):
    # Создаем DataFrame с одним абзацем
    df = pd.DataFrame({'content': [paragraph]})

    # Извлекаем признаки
    features = extract_features(df)

    # Выбираем нужные столбцы
    selected_features = [
        'char_count', 'word_count', 'sentence_count', 'avg_word_length',
        'avg_sentence_length', 'ttr', 'herdan_index', 'yule_index',
        'function_words_ratio', 'avg_punctuation_per_sentence', 'avg_tree_depth',
        'char_entropy', 'word_entropy', 'flesch_kincaid_grade', 'gunning_fog_index',
        'word_repetition_frequency', 'pos_ratio_ADJ', 'pos_ratio_ADP', 'pos_ratio_ADV',
        'pos_ratio_AUX', 'pos_ratio_CCONJ', 'pos_ratio_DET', 'pos_ratio_INTJ',
        'pos_ratio_NOUN', 'pos_ratio_NUM', 'pos_ratio_PART', 'pos_ratio_PRON',
        'pos_ratio_PROPN', 'pos_ratio_PUNCT', 'pos_ratio_SCONJ', 'pos_ratio_SYM',
        'pos_ratio_VERB', 'pos_ratio_X', 'pos_ratio_SPACE'
    ]

    features = features[selected_features]
    #print(features)
    # Классифицируем с помощью каждой модели
    results = {}
    for name, model in models.items():
        prediction = model.predict(features)
        results[name] = prediction[0]

    return results


**Проверяем загрузку моделей и возможность классификации**

In [25]:
# Пример использования
loaded_models1 = load_models1()

Модели загружены.


In [26]:
# Классифицируем произвольный абзац
sample_paragraph = "Пользователи, или клиенты, могут отправлять, получать, просматривать и управлять документами через специальные приложения, которые взаимодействуют с сервером ис учёта ресурсов. Таким образом, ис учёта ресурсов обычно использует клиентсерверную архитектуру, где сервер обрабатывает запросы на поставку и хранение материалов, а клиенты работают с этой системой через свои приложения. Клиентсерверная архитектура в ис учёта ресурсов предприятия обеспечивает удобство и эффективность для пользователей. Клиенты могут использовать специализированные приложения для взаимодействия с сервером, отправлять. Сервер, в свою очередь, обеспечивает централизованное хранение данных, обработку запросов, контроль доступа и безопасность информации."
classification_results = classify_paragraph(sample_paragraph, loaded_models1)

In [27]:
print("Результаты классификации:")
for model_name, prediction in classification_results.items():
    #print(f"{model_name}: {'Позитивный' if prediction == 1 else 'Негативный'}")
    print(f"{model_name}: {prediction}")

Результаты классификации:
RandomForest1: 1
GB1: 1
kNN1: 0
LogisticRegression1: 1
SVC1: 1


________________________**Конец проверки работоспособности моделей на новых признаках** ___________________________

#Создаем интерфейс gradio для работы со всеми моделями

In [28]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.5 MB/s eta 0:00:00


In [29]:
import gradio as gr
from collections import Counter

**Функции для классификации произвольного абзаца текста**

In [30]:
def classify_text(text, model_name):
       try:
           if model_name in loaded_models:
               # Классификация с использованием моделей на tf-idf признаках
               preprocessed_text = preprocess_text(text)
               vectorized_text = loaded_vectorizer.transform([preprocessed_text])
               prediction = loaded_models[model_name].predict(vectorized_text)
           elif model_name in loaded_models1:
               # Классификация с использованием моделей на новых числовых признаках
               df = pd.DataFrame({'content': [paragraph]})
               features = extract_features(df)
               # Выбираем нужные столбцы
               selected_features = [
                   'char_count', 'word_count', 'sentence_count', 'avg_word_length',
                   'avg_sentence_length', 'ttr', 'herdan_index', 'yule_index',
                   'function_words_ratio', 'avg_punctuation_per_sentence', 'avg_tree_depth',
                   'char_entropy', 'word_entropy', 'flesch_kincaid_grade', 'gunning_fog_index',
                   'word_repetition_frequency', 'pos_ratio_ADJ', 'pos_ratio_ADP', 'pos_ratio_ADV',
                   'pos_ratio_AUX', 'pos_ratio_CCONJ', 'pos_ratio_DET', 'pos_ratio_INTJ',
                   'pos_ratio_NOUN', 'pos_ratio_NUM', 'pos_ratio_PART', 'pos_ratio_PRON',
                   'pos_ratio_PROPN', 'pos_ratio_PUNCT', 'pos_ratio_SCONJ', 'pos_ratio_SYM',
                   'pos_ratio_VERB', 'pos_ratio_X', 'pos_ratio_SPACE'
             ]

               features = features[selected_features]
               prediction = loaded_models1[model_name].predict(features)
           else:
               return "Выбрана неизвестная модель"

           return "Имеются признаки сгенерированного текста" if prediction[0] == 1 else "Предположительно текст написан человеком"
       except Exception as e:
           return f"Ошибка: {str(e)}"

# Функция для классификации текста всеми моделями
def classify_all_models(text):
    results = {}
    for model_name in loaded_models:
        results[model_name] = classify_text(text, model_name)
    for model_name in loaded_models1:
        results[model_name] = classify_text(text, model_name)
    return results

# Функция для голосования
def vote_classification(text):
    all_results = classify_all_models(text)
    votes = Counter(all_results.values())
    return votes.most_common(1)[0][0]


**Интерфейс в gradio**

In [31]:
# Создание интерфейса Gradio
def gradio_interface(text, model):
    if model == "все модели":
        results = classify_all_models(text)
        return "\n".join([f"{model}: {result}" for model, result in results.items()])
    elif model == "голосование":
        return vote_classification(text)
    else:
        return classify_text(text, model)

# Список доступных моделей
models = ["все модели", "голосование"] + list(loaded_models.keys()) + list(loaded_models1.keys())

# Создание интерфейса
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(lines=5, label="Введите текст для классификации"),
        gr.Dropdown(choices=models, label="Выберите модель")
    ],
    outputs=gr.Textbox(label="Результат классификации"),
    title="Классификация текста",
    description="Введите текст и выберите модель для классификации."
)

# Запуск интерфейса
iface.launch(share=True)
iface.launch(share=False, server_port=7860)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3702d5bd71586a3dde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

#На всякий случай сделаем еще интерфейс в ipywidgets. Вдруг gradio опять заглючит

In [32]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [33]:
# Список доступных моделей
models = ["все модели", "голосование"] + list(loaded_models.keys()) + list(loaded_models1.keys())

# Создание виджетов
text_input = widgets.Textarea(
    value='',
    placeholder='Введите текст для классификации',
    description='Текст:',
    disabled=False,
    layout={'width': '100%', 'height': '150px'}
)

model_dropdown = widgets.Dropdown(
    options=models,
    value='все модели',
    description='Модель:',
    disabled=False,
)

output = widgets.Output()

# Функция для обработки классификации
def classify(b):
    with output:
        clear_output()
        text = text_input.value
        model = model_dropdown.value

        if model == "все модели":
            results = classify_all_models(text)
            print("\n".join([f"{model}: {result}" for model, result in results.items()]))
        elif model == "голосование":
            print(vote_classification(text))
        else:
            print(classify_text(text, model))

# Создание кнопки
classify_button = widgets.Button(description="Классифицировать")
classify_button.on_click(classify)

# Создание контейнера для виджетов
container = widgets.VBox([
    widgets.HTML("<h2>Классификация текста</h2>"),
    widgets.HTML("<p>Введите текст и выберите модель для классификации.</p>"),
    text_input,
    model_dropdown,
    classify_button,
    output
])

# Отображение интерфейса
display(container)